In [22]:
import os
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
import numpy as np
import matplotlib.pyplot as plt

### We load data here. We also modify the column for better readability. 

In [23]:
df = pd.read_csv("HistoricalData.csv")
df = df.rename(columns={"Default=1": "Default"})
#drop columns
df = df.drop(columns=["Unnamed: 10", "Unnamed: 11"])
# df = df[:4500]

df.head()

,Firm #,E/A,ROA,D/A,OPM,ATR,Size,QR,CR,Default
0,1,0.55,0.43,0.45,0.27,1.59,807.11,1.98,2.12,0
1,2,0.42,0.61,0.58,0.28,2.17,1040.68,1.31,1.87,0
2,3,0.83,0.52,0.17,0.35,1.51,864.26,1.89,2.16,0
3,4,0.51,0.86,0.49,0.32,2.69,1201.17,2.54,2.97,0
4,5,0.57,0.52,0.43,0.24,2.12,1022.42,1.49,2.47,0


In [24]:
# # calculate the mean of the columns for default and non-defult
# mean_default = df[df["Default"] == 1].mean()
# mean_non_default = df[df["Default"] == 0].mean()
# print(mean_default, mean_non_default)

In [25]:
# predicted_df = pd.read_csv("predictions.csv")
# # predicted_df

# # Apply sigmoid to the predicted values
# predicted_df['predicted_prob'] = 1 / (1 + np.exp(-predicted_df['Predicted']))

# # Now predicted_prob will be between 0 and 1
# print(predicted_df[['Predicted', 'predicted_prob', 'Actual']])

### We found that some of the factors are not highly correlated to our outcome, so we are not using them in our model. 

In [26]:
# test the correlation between each column in df
correlation_matrix = df.corr()
print(correlation_matrix)
# it seems that E/A and D/A are highly correlated to each other and we can drop one of them
# also, ATR, Size, and CR are less correlated to Default and we can drop them as well
df = df.drop(columns=["D/A", "ATR", "Size", "CR", "Firm #"])

df.head()

           Firm #       E/A       ROA       D/A       OPM       ATR      Size  \
Firm #   1.000000 -0.013341  0.019614  0.013341  0.017605  0.000017 -0.001784   
E/A     -0.013341  1.000000 -0.322527 -1.000000 -0.196210 -0.250941 -0.399160   
ROA      0.019614 -0.322527  1.000000  0.322527  0.883347  0.091613  0.517479   
D/A      0.013341 -1.000000  0.322527  1.000000  0.196210  0.250941  0.399160   
OPM      0.017605 -0.196210  0.883347  0.196210  1.000000 -0.358687  0.404990   
ATR      0.000017 -0.250941  0.091613  0.250941 -0.358687  1.000000  0.194580   
Size    -0.001784 -0.399160  0.517479  0.399160  0.404990  0.194580  1.000000   
QR      -0.001867  0.196932 -0.000978 -0.196932 -0.087575  0.194861  0.304658   
CR      -0.020892  0.205772  0.028437 -0.205772 -0.101706  0.289805  0.189370   
Default  0.010855 -0.096968 -0.041513  0.096968 -0.047379  0.021253 -0.010492   

               QR        CR   Default  
Firm #  -0.001867 -0.020892  0.010855  
E/A      0.196932  0.205772 

,E/A,ROA,OPM,QR,Default
0,0.55,0.43,0.27,1.98,0
1,0.42,0.61,0.28,1.31,0
2,0.83,0.52,0.35,1.89,0
3,0.51,0.86,0.32,2.54,0
4,0.57,0.52,0.24,1.49,0


### Define our model
### We also get our own predicted default values from the model
### We also see the coefficients of the model

In [27]:
# predict model using linear regression 
# x is all columns except Default and y is Default
# using the first 4500 rows of the dataset for training and the rest for testing
x = df[['E/A','ROA','OPM','QR']]
y = df['Default']
model = LogisticRegression()
model.fit(x, y)
# # generate the predicted values and store them in a new column called Predicted
# df['Predicted'] = model.predict(x)
# # apply sigmoid to the predicted values
# df['Predicted'] = 1 / (1 + np.exp(-df['Predicted']))
# df
# # model.coef_
# # model.intercept_

predicted_probs = model.predict_proba(x)[:, 1]  # Probability of Default (class=1)

# Attach to your data
df['predicted_prob'] = predicted_probs

# View some results
print(df[['predicted_prob', 'Default']].head())
# print each coefficient of the model
for feature, coef in zip(x.columns, model.coef_[0]):
    print(f"Feature: {feature}, Coefficient: {coef}")
print("Model Coefficients: ", model.coef_)
print("Model Intercept: ", model.intercept_)


   predicted_prob  Default
0        0.053970        0
1        0.102086        0
2        0.019226        0
3        0.026623        0
4        0.064827        0
Feature: E/A, Coefficient: -3.399888994876017
Feature: ROA, Coefficient: -1.0406879874456771
Feature: OPM, Coefficient: -1.0299288035961345
Feature: QR, Coefficient: -0.6645227895179829
Model Coefficients:  [[-3.39988899 -1.04068799 -1.0299288  -0.66452279]]
Model Intercept:  [1.04741505]


### We find the trend in the actual default data and the predicted default data

In [28]:
# print all the rows where "Default" is 1
risk = df[df["Default"] == 1]
print(risk)
# what is the minimum value of predicted for risk
# print(risk["Predicted"].min())
# print a summary of the risk
print(risk.describe())

       E/A   ROA   OPM    QR  Default  predicted_prob
11    0.30  0.78  0.40  1.68        1        0.090080
19    0.46  0.87  0.53  1.84        1        0.039524
22    0.53  0.65  0.32  1.96        1        0.044659
31    0.71  0.38  0.18  1.67        1        0.044912
127   0.63  0.64  0.30  1.73        1        0.038451
...    ...   ...   ...   ...      ...             ...
4882  0.71  0.42  0.24  2.02        1        0.032511
4907  0.44  0.69  0.30  1.47        1        0.079260
4936  0.54  0.56  0.23  2.09        1        0.047559
4967  0.60  0.57  0.30  1.99        1        0.038528
4980  0.60  0.94  0.46  2.54        1        0.015791

[211 rows x 6 columns]
              E/A         ROA         OPM          QR  Default  predicted_prob
count  211.000000  211.000000  211.000000  211.000000    211.0      211.000000
mean     0.552085    0.550521    0.276398    1.873460      1.0        0.055538
std      0.106839    0.193758    0.100349    0.385535      0.0        0.026790
min      0.2

In [29]:
# assign each coefficient to a variable from the model
a = model.intercept_[0]
b1 = model.coef_[0][0]
b2 = model.coef_[0][1]
b3 = model.coef_[0][2]
b4 = model.coef_[0][3]
print(a, b1, b2, b3, b4)

# Feature: E/A, Coefficient: -3.399888994876017
# Feature: ROA, Coefficient: -1.0406879874456771
# Feature: OPM, Coefficient: -1.0299288035961345
# Feature: QR, Coefficient: -0.6645227895179829


1.0474150520739245 -3.399888994876017 -1.0406879874456771 -1.0299288035961345 -0.6645227895179829


### Load new data

In [55]:
# load "Competition data.csv"
df2 = pd.read_csv("Competition data.csv")
df2

,Client,Max Willing to Pay Interest Rate,Bidding Offer,Default,E/A,ROA,D/A,OPM,ATR,Size,QR,CR
0,Client 1,0.07,NaN,0,0.62,0.65,0.38,0.30,2.14,1040.22,2.03,2.51
1,Client 2,0.05,NaN,0,0.64,0.91,0.36,0.55,1.67,1015.47,1.56,2.45
2,Client 3,0.07,NaN,0,0.64,0.92,0.36,0.50,1.82,893.21,1.49,2.34
3,Client 4,0.12,NaN,0,0.16,0.50,0.84,0.20,2.52,1051.25,2.80,2.41
4,Client 5,0.07,NaN,0,0.61,0.57,0.39,0.28,2.02,990.66,2.06,2.41
...,...,...,...,...,...,...,...,...,...,...,...,...
495,Client 496,0.12,NaN,0,0.31,0.61,0.69,0.29,2.11,917.03,1.56,1.78
496,Client 497,0.05,NaN,0,0.72,0.75,0.28,0.40,1.85,943.02,2.08,3.13
497,Client 498,0.09,NaN,0,0.76,0.35,0.24,0.21,1.72,843.28,2.58,2.44
498,Client 499,0.09,NaN,0,0.54,0.55,0.46,0.35,1.59,927.16,2.26,2.84


In [56]:
# for row in df2.iterrows():
#    EA = df2.loc[row[0], "E/A"]
#    ROA = df2.loc[row[0], "ROA"]
#    OPM = df2.loc[row[0], "OPM"]
#    QR = df2.loc[row[0], "QR"]
#    # calculate the predicted value using the model
#    predicted = a + b1 * EA + b2 * ROA + b3 * OPM + b4 * QR
#    # print(predicted)
#    # add the predicted value to the dataframe
#    df2.loc[row[0], "Predicted"] = predicted
x = df2[['E/A','ROA','OPM','QR']]
y = df2['Default']
predicted_probs = model.predict_proba(x)[:, 1]
df2['predicted_prob'] = predicted_probs

risk2 = df2[df2["Default"] == 1]
risk2[risk2["predicted_prob"] > 0.05].describe()
# risk2.describe()

,Max Willing to Pay Interest Rate,Bidding Offer,Default,E/A,ROA,D/A,OPM,ATR,Size,QR,CR,predicted_prob
count,25.000000,0.0,25.0,25.000000,25.000000,25.000000,25.000000,25.00000,25.000000,25.000000,25.000000,25.000000
mean,0.124000,NaN,1.0,0.292800,0.485600,0.707200,0.236000,2.08920,980.374400,1.812800,2.180800,0.139688
std,0.023979,NaN,0.0,0.148535,0.153869,0.148535,0.078846,0.27383,96.141358,0.371702,0.466886,0.060104
min,0.090000,NaN,1.0,0.070000,0.170000,0.430000,0.100000,1.61000,722.510000,1.260000,1.430000,0.057914
25%,0.100000,NaN,1.0,0.180000,0.390000,0.600000,0.170000,1.89000,927.000000,1.480000,1.960000,0.098795
50%,0.130000,NaN,1.0,0.270000,0.480000,0.730000,0.220000,2.04000,968.420000,1.820000,2.200000,0.131365
75%,0.140000,NaN,1.0,0.400000,0.600000,0.820000,0.300000,2.24000,1052.160000,2.120000,2.460000,0.178933
max,0.160000,NaN,1.0,0.570000,0.810000,0.930000,0.410000,2.63000,1149.530000,2.480000,3.060000,0.310092


In [57]:
# reassign values to "Default" column
df2.loc[df2["predicted_prob"] > 0.05, "Default"] = 1
df2.loc[df2["predicted_prob"] <= 0.05, "Default"] = 0
df2

,Client,Max Willing to Pay Interest Rate,Bidding Offer,Default,E/A,ROA,D/A,OPM,ATR,Size,QR,CR,predicted_prob
0,Client 1,0.07,NaN,0,0.62,0.65,0.38,0.30,2.14,1040.22,2.03,2.51,0.032455
1,Client 2,0.05,NaN,0,0.64,0.91,0.36,0.55,1.67,1015.47,1.56,2.45,0.024635
2,Client 3,0.07,NaN,0,0.64,0.92,0.36,0.50,1.82,893.21,1.49,2.34,0.026830
3,Client 4,0.12,NaN,1,0.16,0.50,0.84,0.20,2.52,1051.25,2.80,2.41,0.110705
4,Client 5,0.07,NaN,0,0.61,0.57,0.39,0.28,2.02,990.66,2.06,2.41,0.036368
...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,Client 496,0.12,NaN,1,0.31,0.61,0.69,0.29,2.11,917.03,1.56,1.78,0.121671
496,Client 497,0.05,NaN,0,0.72,0.75,0.28,0.40,1.85,943.02,2.08,3.13,0.018430
497,Client 498,0.09,NaN,0,0.76,0.35,0.24,0.21,1.72,843.28,2.58,2.44,0.021217
498,Client 499,0.09,NaN,0,0.54,0.55,0.46,0.35,1.59,927.16,2.26,2.84,0.038302


In [58]:
type(df2["Max Willing to Pay Interest Rate"][0])

numpy.float64

In [59]:
# for "Default" == 1, "Bidding Offer" is "NaN"
df2.loc[df2["Default"] == 1, "Bidding Offer"] = np.nan
# for "Default" == 0, find "Clinet" whose "Max Willing to Pay Interest Rate" is between 9% and 5%, set "Bidding Offer" equal to "Max Willing to Pay Interest Rate"
df2.loc[(df2["Default"] == 0) & (df2["Max Willing to Pay Interest Rate"] <= 0.10) & (df2["Max Willing to Pay Interest Rate"] >= 0.07), "Bidding Offer"] = df2["Max Willing to Pay Interest Rate"]


In [60]:
# count how many rows have "Bidding Offer" == 0
df2[df2["Bidding Offer"] == 0].count()
# count how many rows have "Bidding Offer" is empty
df2[df2["Bidding Offer"].isnull()].count()
# count how many rows have "Bidding Offer" is not empty and not 0
df2[df2["Bidding Offer"].notnull() & (df2["Bidding Offer"] != 0)].count()

Client                              170
Max Willing to Pay Interest Rate    170
Bidding Offer                       170
Default                             170
E/A                                 170
ROA                                 170
D/A                                 170
OPM                                 170
ATR                                 170
Size                                170
QR                                  170
CR                                  170
predicted_prob                      170
dtype: int64

In [61]:
# This part sorts the dataframe by "predicted_prob" in ascending order and let us know which clients we are paying the most attention to

# create a sub-dataframe using "clients" whose "Bidding Offer" is not empty and not 0
df3 = df2[df2["Bidding Offer"].notnull() & (df2["Bidding Offer"] != 0)]
# sort by "predicted_prob" in ascending order
df3 = df3.sort_values(by="predicted_prob", ascending=True)
df3

,Client,Max Willing to Pay Interest Rate,Bidding Offer,Default,E/A,ROA,D/A,OPM,ATR,Size,QR,CR,predicted_prob
374,Client 375,0.08,0.08,0,0.80,0.87,0.20,0.40,2.18,1117.77,2.62,3.25,0.008741
134,Client 135,0.08,0.08,0,0.82,0.79,0.18,0.42,1.87,1035.03,2.55,2.34,0.009105
479,Client 480,0.07,0.07,0,0.80,0.98,0.20,0.53,1.86,1037.03,2.15,2.63,0.009313
137,Client 138,0.07,0.07,0,0.84,0.93,0.16,0.43,2.17,934.97,2.16,2.82,0.009428
264,Client 265,0.07,0.07,0,0.85,0.64,0.15,0.40,1.62,999.44,2.59,2.47,0.009550
...,...,...,...,...,...,...,...,...,...,...,...,...,...
285,Client 286,0.10,0.10,0,0.59,0.66,0.41,0.33,2.03,867.11,1.54,1.47,0.047038
94,Client 95,0.10,0.10,0,0.61,0.59,0.39,0.26,2.28,939.44,1.63,3.11,0.047812
328,Client 329,0.10,0.10,0,0.54,0.33,0.46,0.17,1.93,1083.94,2.52,3.21,0.048262
73,Client 74,0.10,0.10,0,0.58,0.61,0.42,0.29,2.10,938.19,1.69,1.52,0.048289


In [62]:
## if "loan" = 0, it means that we are offering this client a loan, and we don't need to borrow money to give them a loan
## if "loan" = 1, it means that we are borrowing money to give this client a loan, and we need to borrow money to give them a loan
## if "loan" = 2, it means that we are not offering this client a loan because they are a default client
## if "loan" = NaN, it means that we are not considering this client at this time


# use the index of first 25 rows of df3, assign the same index to df2 and create a new column "loan" in df2
df2["loan"] = 1
df2.loc[df3.index[:25], "loan"] = 0
# change the value of "loan" to 2 for "Default" == 1
df2.loc[df2["Default"] == 1, "loan"] = 2
# change the value of "loan" to "NaN" fir "Bidding Offer" == "NaN"
df2.loc[df2["Bidding Offer"].isnull(), "loan"] = np.nan
df2

,Client,Max Willing to Pay Interest Rate,Bidding Offer,Default,E/A,ROA,D/A,OPM,ATR,Size,QR,CR,predicted_prob,loan
0,Client 1,0.07,0.07,0,0.62,0.65,0.38,0.30,2.14,1040.22,2.03,2.51,0.032455,1.0
1,Client 2,0.05,NaN,0,0.64,0.91,0.36,0.55,1.67,1015.47,1.56,2.45,0.024635,NaN
2,Client 3,0.07,0.07,0,0.64,0.92,0.36,0.50,1.82,893.21,1.49,2.34,0.026830,1.0
3,Client 4,0.12,NaN,1,0.16,0.50,0.84,0.20,2.52,1051.25,2.80,2.41,0.110705,NaN
4,Client 5,0.07,0.07,0,0.61,0.57,0.39,0.28,2.02,990.66,2.06,2.41,0.036368,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,Client 496,0.12,NaN,1,0.31,0.61,0.69,0.29,2.11,917.03,1.56,1.78,0.121671,NaN
496,Client 497,0.05,NaN,0,0.72,0.75,0.28,0.40,1.85,943.02,2.08,3.13,0.018430,NaN
497,Client 498,0.09,0.09,0,0.76,0.35,0.24,0.21,1.72,843.28,2.58,2.44,0.021217,1.0
498,Client 499,0.09,0.09,0,0.54,0.55,0.46,0.35,1.59,927.16,2.26,2.84,0.038302,1.0


In [63]:
#export df2 to csv 
# this file contains the final clients we are offering loans to
df2.to_csv("Competition data 2.csv", index=False)


In [21]:
# reload "Competition data 2.csv" as a new dataframe

